<small><small>**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.</small></small>



# Imports

## Equality

In [ ]:
module code.eq where

--import Relation.Binary.PropositionalEquality as Eq
--open Eq using (_≡_; refl; trans; sym; cong; cong-app; subst) public
--open Eq.≡-Reasoning using (begin_; _≡⟨⟩_; _≡⟨_⟩_; _∎) public

infix 4 _≡_
data _≡_ {A : Set} (x : A) : A → Set where
  refl : x ≡ x
  
sym : {A : Set} {x y : A} → x ≡ y → y ≡ x
sym refl = refl

trans : {A : Set} {x y z : A} → x ≡ y → y ≡ z → x ≡ z
trans refl refl = refl

cong : {A B : Set} (f : A → B) {x y : A} → x ≡ y → f x ≡ f y
cong f refl = refl

cong2 : {A B C : Set} (f : A → B → C) {a1 a2 : A} {b1 b2 : B} →
    a1 ≡ a2 →
    b1 ≡ b2 →
    f a1 b1 ≡ f a2 b2
cong2 f refl refl = refl

subst : {A : Set} {x y : A} (P : A → Set) → x ≡ y → P x → P y
subst P refl px = px

-- equational reasoning
infix  1 begin_
infixr 2 _≡⟨⟩_ _≡⟨_⟩_
infix  3 _∎

begin_ : {A : Set} {x y : A} → x ≡ y → x ≡ y
begin x≡y = x≡y

_≡⟨⟩_ : {A : Set} (x : A) {y : A} → x ≡ y → x ≡ y
x ≡⟨⟩ x≡y = x≡y

_≡⟨_⟩_ : {A : Set} (x : A) {y z : A} → x ≡ y → y ≡ z → x ≡ z
x ≡⟨ x≡y ⟩ y≡z = trans x≡y y≡z

_∎ : {A : Set} (x : A) → x ≡ x
x ∎ = refl

## Booleans

In [ ]:
module code.boolean where

data 𝔹 : Set where
  true : 𝔹
  false : 𝔹

-- equality on booleans
infix 10 _≡𝔹_
data _≡𝔹_ : 𝔹 → 𝔹 → Set where
  tt : true ≡𝔹 true
  ff : false ≡𝔹 false

## Quantifiers

In [3]:
module code.existential where

--data Σ {l} (A : Set l) (B : A → Set) : Set l where
--    ⟨_,_⟩ : (a : A) → B a → Σ A B

record Σ {l} (A : Set l) (B : A → Set) : Set l where
    constructor ⟨_,_⟩
    field
      proj₁ : A
      proj₂ : B proj₁

infix 0 thereExists
thereExists : ∀ {l} {A : Set l} (B : A → Set) → Set l
thereExists {_} {A} B = Σ A B

syntax thereExists (λ x → B) = ∃[ x ] B

In [4]:
module code.universal where

Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a

apply : {A : Set} → {B : A → Set} → Π A B → (a : A) → B a
apply f x = f x

infix 0 forAll
forAll : {A : Set} → (B : A → Set) → Set
forAll {A} B = Π A B

syntax forAll (λ x → B) = ∀[ x ] B

## Lists

In [ ]:
module code.list where
open import code.conn public
open import code.nat public

infixr 5 _∷_
data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *

length : {A : Set} → A * → ℕ
length ε = 0
length (_ ∷ xs) = suc (length xs)

infix 4 _≡L_
data _≡L_ {A : Set} : A * → A * → Set where
  ε≡ε : ε ≡L ε
  ∷≡∷ : {a : A} {as bs : A *} → as ≡L bs → a ∷ as ≡L a ∷ bs

≡L-refl : {A : Set} {as : A *} → as ≡L as
≡L-refl {A} {ε} = ε≡ε
≡L-refl {A} {a ∷ as} = ∷≡∷ (≡L-refl {A} {as})

≡L-sym : {A : Set} {as bs : A *} → as ≡L bs → bs ≡L as
≡L-sym ε≡ε = ε≡ε
≡L-sym {A} (∷≡∷ {a} {as} {bs} as≡bs) = ∷≡∷ (≡L-sym {A} {as} {bs} as≡bs)

postulate ≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs

## Logical connectives

In [ ]:
module code.conn where -- from before

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B

fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B
    
case : {A B C : Set} → A ∨ B → (A → C) → (B → C) → C
case (left a) f g = f a
case (right b) f g = g b

data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥

infixr 1 _⟺_
_⟺_ : (A : Set) (B : Set) → Set
A ⟺ B = (A → B) ∧ (B → A) 

## Natural numbers

In [ ]:
module code.nat where

data ℕ : Set where 
  zero : ℕ
  suc : ℕ → ℕ
  
{-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_
infixl 6 _*_ -- higher priority than _+_

_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

_*_ : ℕ → ℕ → ℕ
zero * _  =  zero
suc m * n  =  n + (m * n)

infix 4 _≡ℕ_

-- indexed inductive type
data _≡ℕ_ : ℕ → ℕ → Set where
  z≡z : 0 ≡ℕ 0
  s≡s : {m n : ℕ} → m ≡ℕ n → suc m ≡ℕ suc n
  
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n

# Finite sets

````
module code.fin where
open import code.nat public
````

We define the set of all natural numbers smaller than a given natural number.

````
data Fin : ℕ → Set where
  ∅ : Fin 0
  next : {n : ℕ} → Fin n → Fin (suc n)
````

Thus `∅` is the empty set which contains no number,
and `Fin n` contains the numbers $n - 1$, $n - 2$, $\dots$, $0$.

# Vectors

## Lists and external verification

We have previously see how one can define lists as an inductive data type:

````
module code.list2 where
open import code.nat public

data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *
````

One can define recursive functions on lists,
such as concatenation and length.

````
_++_ : {A : Set} → A * → A * → A *
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)

length : {A : Set} → A * → ℕ
length ε = 0
length (_ ∷ xs) = suc (length xs)
````

Then one can prove that concatenation preserves the combined length of the two lists (proof omitted).

````
postulate ++-len : {A : Set} → (as bs : A *) → length (as ++ bs) ≡ℕ length as + length bs
````

This is an example of *external verification*,
whereby the definition of `_++_`, `length`
are given separately from the property `++-len` they satisfy.

## Vectors and internal verification

````
module code.vector where
open import code.nat public
open import code.boolean public
open import code.existential public
open import code.conn public
````

An alternative approach is offered by *internal verification*,
whereby we encode the length of the list inside its type,
obtaining *vectors*.

````
infixr 5 _∷_

data Vector (A : Set) : ℕ → Set where
  ε : Vector A zero
  _∷_ : {n : ℕ} → A → Vector A n → Vector A (suc n)
````

The idea is that a vector of type `Vector A n` has length n.
The type `Vector A n` is *parameterised* by `A` (since `A` is constant in the inductive definition)
and indexed by the dimension `n` (since `n` varies )

We can immediately transform a vector into a list,
by just ignoring the information about the dimension.

````
open import code.list public -- hiding (_++_)

toList : {A : Set} {n : ℕ} → Vector A n → A *
toList ε = ε
toList (x ∷ xs) = x ∷ toList xs
````

**Exercise.** Define the converse transformation from lists to vectors.

*Hint:* Given a list `xs`, for which `n` the corresponding vector has type `Vector A n`?

In [ ]:
module code.vector.fromList where
open import code.vector public

fromList : {A : Set} → (xs : A *) → ?
fromList = ?

?0 : _1
?1 : (xs : A *) → ?0 (xs = xs)
Sort _1  [ at /Users/lorenzo/Development/agda-kernel/example/code/vector/fromList.agda:4,37-38 ]


In [12]:
module code.vector.fromList where
open import code.vector public

fromList : {A : Set} → (xs : A *) → Vector A (length xs)
fromList ε = ε
fromList (x ∷ xs) = x ∷ fromList xs

## Concatenation of vectors

Like for lists, we can define concatenation of vectors.
 
````
module code.vector.concat where
open import code.vector public
infixr 5 _++_

_++_ : {A : Set} {m n : ℕ} → Vector A m → Vector A n → Vector A (m + n)
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)
````

In [ ]:
-- The following example shows that there are problems to show associativity
-- of concatenation of vectors

module code.vector.concat.assoc where
open import code.vector.concat public
open import code.eq public

-- cannote even state associativity,
-- because the types don't match:
-- need a proof to show that
-- Vector A (m + n + o) ≡ Vector A ((m + n) + o)

{-
++-assoc : {A : Set} {m n o : ℕ} →
    (xs : Vector A m) →
    (ys : Vector A n) →
    (zs : Vector A o) →
    xs ++ ys ++ zs ≡ (xs ++ ys) ++ zs
++-assoc = ?
-}

## Map on vectors

**Exercise.** Define the map function on vectors.

In [ ]:
module code.vector.map where
open import code.vector public

map : {A B : Set} {n : ℕ} → (A → B) → ? → ?
map f = ?

?0 : _3
?1 : _5
?2 : ?0 → ?1
Sort _3  [ at /Users/lorenzo/Development/agda-kernel/example/code/vector/map.agda:4,39-40 ]
Sort _5  [ at /Users/lorenzo/Development/agda-kernel/example/code/vector/map.agda:4,43-44 ]


In [ ]:
module code.vector.map where
open import code.vector public

map : {A B : Set} {n : ℕ} → (A → B) → Vector A n → Vector B n
map f ε = ε
map f (x ∷ xs) = f x ∷ map f xs

## Length of a vector

**Exercise.** Define a function that extracts the length of a given vector.
*Hint:* We need to look at the implicit parameter `n`.

In [ ]:
module code.vector.length where
open import code.vector public

vlength : {A : Set} {n : ℕ} → Vector A n → ℕ
vlength = ?

?0 : Vector A n → ℕ


In [18]:
module code.vector.length where
open import code.vector public

-- need to access the hidden parameter!
vlength : {A : Set} {n : ℕ} → Vector A n → ℕ
vlength {A} {n} _ = n

## Filter on vectors

Filtering a vector is a more challenging task,
since it does not preserve the length of the list.

**Exercise.** In a first approximation, we just want to prove that the filter function returns a vector of *some length*. Complete the definition below.

*Hint:* Recall that a proof of `∃[ n ] Vector A n` is a pair `⟨ n , bs ⟩`
where `n : ℕ` and `bs : Vector A n`.

In [ ]:
module code.vector.filter where
open import code.vector public

filter : {A : Set} {m : ℕ} → (A → 𝔹) → Vector A m → ∃[ n ] Vector A n
filter _ ε = ⟨ ? , ? ⟩
filter p (a ∷ as) with p a | filter p as
... | true | ⟨ n , bs ⟩ = ⟨ ? , ? ⟩
... | false | ⟨ n , bs ⟩ = ⟨ ? , ? ⟩

?0 : ℕ
?1 : Vector A ?0
?2 : ℕ
?3 : Vector A ?2
?4 : ℕ
?5 : Vector A ?4


In [ ]:
module code.vector.filter where
open import code.vector public

filter : {A : Set} {m : ℕ} → (A → 𝔹) → Vector A m → ∃[ n ] Vector A n
filter _ ε = ⟨ zero , ε ⟩
filter p (a ∷ as) with p a | filter p as
... | true | ⟨ n , bs ⟩ = ⟨ suc n , a ∷ bs ⟩
... | false | ⟨ n , bs ⟩ = ⟨ n , bs ⟩

**Exercise**. Refine the solution above and prove that the filter function cannot increase the length.

*Hint:* A proof of `∃[ n ] n ≤ m ∧ Vector A n` has the shape
`⟨ n , ⟨ n≤m , bs ⟩ ⟩`.

In [ ]:
module code.vector.filter where
open import code.vector public

postulate ≤-suc : {m n : ℕ} → m ≤ n → m ≤ suc n

filter : {A : Set} {m : ℕ} → (A → 𝔹) → Vector A m → ∃[ n ] n ≤ m ∧ Vector A n
filter = ?

?0
  : (A → 𝔹) → Vector A m → thereExists (λ n → (n ≤ m) ∧ Vector A n)


In [ ]:
module code.vector.filter where
open import code.vector public

postulate ≤-suc : {m n : ℕ} → m ≤ n → m ≤ suc n

filter : {A : Set} {m : ℕ} → (A → 𝔹) → Vector A m → ∃[ n ] n ≤ m ∧ Vector A n
filter _ ε = ⟨ 0 , ⟨ 0≤n , ε ⟩ ⟩
filter {A} {suc m} p (a ∷ as) with p a | filter p as
... | true | ⟨ n , ⟨ n≤m , bs ⟩ ⟩ = ⟨ suc n , ⟨ s≤s n≤m , a ∷ bs ⟩ ⟩
... | false | ⟨ n , ⟨ n≤m , bs ⟩ ⟩ = ⟨ n , ⟨ ≤-suc n≤m , bs ⟩ ⟩

##  Vectors of vectors

**Exercise.** Define a function that concatenates a vector of vectors into a single vector.

In [ ]:
module code.vector.vectors where
open import code.vector.concat public

concat : {A : Set} {m n : ℕ} → Vector (Vector A n) m → Vector A (m * n)
concat {m = 0} ε = ?
concat {m = suc m} (xs ∷ xss) = ?

?0 : Vector A (zero * n)
?1 : Vector A (suc m * n)


In [ ]:
module code.vector.vectors where
open import code.vector.concat public

concat : {A : Set} {m n : ℕ} → Vector (Vector A n) m → Vector A (m * n)
concat {m = 0} ε = ε
concat {m = suc m} (xs ∷ xss) = xs ++ concat xss

## Indexing

In Agda we can only write total functions. (Not all of them.)
Consider the signature for the element-at function.

```agda
-- _!_ : {n : ℕ} {A : Set} → Vector A n → ℕ → A
```

This function as it stands cannot be written in Agda
because it is not clear what to do when `i` exceeds `n`.

### Default values

In order to get out of trouble we will put more constraints on the user of the function.
One way is to require the user to provide a default value to be returned in case of error

**Exercise.** Define indexing with a default answer.

In [ ]:
module code.vector.index where
open import code.vector public

_!_default_ : {n : ℕ} {A : Set} → Vector A n → ℕ → A → A
ε ! _ default a = a
(a ∷ _) ! 0 default _ = a
(_ ∷ as) ! (suc n) default a = as ! n default a

### Bounded index

Another way out is to require the user to prove that the index is smaller than the length of the vector.

**Exercise.** Define indexing by forcing the user to provide evidence that the index is not too large. Notice that we start counting the elements from zero.

In [26]:
module code.vector.index where
open import code.vector public

_!_ : {n m : ℕ} {A : Set} → Vector A n → suc m ≤ n → A
(a ∷ _) ! (s≤s 0≤n) = a
(_ ∷ as) ! (s≤s (s≤s x)) = as ! (s≤s x)

## Matrices

````
module code.matrix where
open import code.vector public
````

A matrix of dimension $m \times n$ is just a vector of dimension $m$
containing vectors of dimension $n$.

````
Matrix : Set → ℕ → ℕ → Set
Matrix A m n = Vector (Vector A n) m
````

In [ ]:
module code.matrix.transpose where
open import code.matrix public

vec : {n : ℕ} {A : Set} → A → Vector A n
vec {zero} _ = ε
vec {suc n} a = a ∷ vec a

zipWith : ∀ {A B C n} → (A → B → C) → Vector A n → Vector B n → Vector C n
zipWith _ ε ε = ε
zipWith f (a ∷ as) (b ∷ bs) = f a b ∷ zipWith f as bs

apply : ∀ {A B : Set} → (A → B) → A → B
apply f x = f x

infixl 90 _$_
_$_ : ∀ {A B n} → Vector (A → B) n → Vector A n → Vector B n
fs $ xs = zipWith apply fs xs

transpose : ∀ {A m n} → Matrix A m n → Matrix A n m
transpose {m = zero} _ = vec ε
transpose {m = suc m} (xs ∷ xss) = zipWith _∷_ xs (transpose xss)

-- transpose-involution : ∀ {A m n} {xss : Matrix A m n} → transpose (transpose xss) ≡ xss

# Decidables

## Order on `ℕ`

Consider again the inductive definition of `≤` for natural numbers:

```agda
--data _≤_ : ℕ → ℕ → Set where
--0≤n : {n : ℕ} → 0 ≤ n
--s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
```

<!--The essence of this definition is that evidence of `m ≤ n`
can be used to produce evidence of `suc m ≤ suc n`. -->

While the definition of `≤` can be used to establish `m ≤ n`
for concrete numbers `m` and `n`,
it does not directly yield an algorithm for checking whether `m ≤ n` holds.

(And for a good reason. As we will see below, inductive definitions such as `≤` can encode two-counter Minsky machines, and therefore such relations are undecidable in general.)

**Exercise.** Write a program that returns `true` or `false` depending on whether `m ≤ n` holds or not.

*Hint:* Proceed by recursion on the first argument.

In [ ]:
module code.bool.leq where
open import code.nat public
open import code.boolean public

_≤ᵇ_ : ℕ → ℕ → 𝔹
m ≤ᵇ n = ?

?0 : 𝔹


In [ ]:
module code.bool.leq where
open import code.nat public
open import code.boolean public
open import code.conn public

infix 15 _≤ᵇ_
_≤ᵇ_ : ℕ → ℕ → 𝔹
0 ≤ᵇ _ = true
(suc _) ≤ᵇ 0 = false
(suc m) ≤ᵇ (suc n) = m ≤ᵇ n

**Exercise.** Show that the function `_≤ᵇ_` is correct.

In [31]:
module code.bool.leq.correct where
open import code.bool.leq public

≤ᵇ→≤ : (m n : ℕ) → m ≤ᵇ n ≡𝔹 true → m ≤ n
≤ᵇ→≤ 0 _  _ = ?
≤ᵇ→≤ (suc m) (suc n) t = ?

≤→≤ᵇ : {m n : ℕ} → m ≤ n → m ≤ᵇ n ≡𝔹 true
≤→≤ᵇ 0≤n = ?
≤→≤ᵇ (s≤s m≤n) = ?

?0 : zero ≤ n
?1 : suc m ≤ suc n
?2 : 0 ≤ᵇ n ≡𝔹 true
?3 : suc m ≤ᵇ suc n ≡𝔹 true


In [32]:
module code.bool.leq.correct where
open import code.bool.leq public

≤ᵇ→≤ : (m n : ℕ) → m ≤ᵇ n ≡𝔹 true → m ≤ n
≤ᵇ→≤ 0 _  _ = 0≤n
≤ᵇ→≤ (suc m) 0 ()
≤ᵇ→≤ (suc m) (suc n) t = s≤s (≤ᵇ→≤ m n t)

≤→≤ᵇ : {m n : ℕ} → m ≤ n → m ≤ᵇ n ≡𝔹 true
≤→≤ᵇ 0≤n = tt
≤→≤ᵇ (s≤s m≤n) = ≤→≤ᵇ m≤n

The function `_≤ᵇ_` *computes* the answer to whether `m ≤ n` holds.
This means that `m ≤ n` is a *decidable* property of the natural numbers.

However, `_≤ᵇ_` does not itself compute a *proof* that `m ≤ n` holds,
which is what we established separately in the exercise above (external verification).

Depending on the context, it might be more convenient to use `_≤ᵇ_` or `_≤_`.
For instance, if we wanted to prove transitivity, then `_≤_` is more convenient because we can do induction on the evidence that `m ≤ n` holds.

On the other hand, if we wanted to define the min function on the natural numbers, then `_≤ᵇ_` is more convenient because we are just interested to know which one is the smaller number, but not why it is so.

## Decidable properties

````
module code.decidable where
open import code.conn public
````

We can combine a decision procedure for a property
together with the construction of the evidence that the property holds
with the introduction of the class of *decidable properties*.

````
data Dec (A : Set) : Set where
  yes : A → Dec A
  no : ¬ A → Dec A
````

The intuition is that evidence for `Dec A` has one of two possible forms:
Either it is of the form `yes p` where `p` is a proof that `A` holds,
or it is of the form `no ¬p` where `¬p` is a proof that `¬ A` holds.

The crucial improvement is the introduction of evidence also when the property does not hold.

**Exercise.** Show that the order `_≤_` on natural number is decidable.
Notice how this is an instance of *internal verification*.

In [34]:
module code.nat.leq.decidable where
open import code.decidable public
open import code.nat public

≤-suc : {m n : ℕ} → suc m ≤ suc n → m ≤ n
≤-suc = ?

_≤?_ : (m n : ℕ) → Dec (m ≤ n)
0 ≤? _ = ?
(suc _) ≤? 0 = ?
(suc m) ≤? (suc n) with m ≤? n
... | yes m≤n = ?
... | no ¬m≤n = ?

?0 : suc m ≤ suc n → m ≤ n
?1 : Dec (zero ≤ n)
?2 : Dec (suc x ≤ zero)
?3 : Dec (suc m ≤ suc n)
?4 : Dec (suc m ≤ suc n)


In [35]:
module code.nat.leq.decidable where
open import code.decidable public
open import code.nat public

≤-suc : {m n : ℕ} → suc m ≤ suc n → m ≤ n
≤-suc (s≤s m≤n) = m≤n

_≤?_ : (m n : ℕ) → Dec (m ≤ n)
0 ≤? _ = yes 0≤n
(suc _) ≤? 0 = no λ ()
(suc m) ≤? (suc n) with m ≤? n
... | yes m≤n = yes (s≤s m≤n)
... | no ¬m≤n = no λ sucm≤sucn → ¬m≤n (≤-suc sucm≤sucn)

## The logic of decidable properties

````
module code.decidable.logic where
open import code.decidable public
open import code.existential public
open import code.universal public
````

One can ask which closure properties do decidable properties have.
It turns out, that decidable properties obey the rules of classical logic.

### Negation

**Exercise.** Show that decidable properties are closed under negation.

*Hint:* Swap the `yes` and the `no` case.

In [ ]:
module code.decidable.neg where
open import code.decidable public

¬? : {A : Set} → Dec A → Dec (¬ A)
¬? (yes x)  =  no λ ¬x → ¬x x
¬? (no ¬x)  =  yes ¬x

### Conjunction

**Exercise.** Show that decidable properties are closed under conjunction.

In [ ]:
module code.decidable.and where
open import code.decidable public

_∧?_ : {A B : Set} → Dec A → Dec B → Dec (A ∧ B)
(yes a) ∧? (yes b) = ?
(no ¬a) ∧? _ = ?
_ ∧? (no ¬b) = ?

?0 : Dec (A ∧ B)
?1 : Dec (A ∧ B)
?2 : Dec (A ∧ B)


In [ ]:
module code.decidable.and where
open import code.decidable public

_∧?_ : {A B : Set} → Dec A → Dec B → Dec (A ∧ B)
(yes a) ∧? (yes b) = yes ⟨ a , b ⟩
(no ¬a) ∧? _ = no λ a∧b → ¬a (fst a∧b)
_ ∧? (no ¬b) = no λ a∧b → ¬b (snd a∧b)

### Disjunction

**Exercise.** Show that decidable properties are closed under disjunction.

In [ ]:
module code.decidable.or where
open import code.decidable public

_∨?_ : {A B : Set} → Dec A → Dec B → Dec (A ∨ B)
_ ∨? _ = ?

?0 : Dec (A ∨ B)


In [ ]:
module code.decidable.or where
open import code.decidable public

_∨?_ : {A B : Set} → Dec A → Dec B → Dec (A ∨ B)
yes a ∨? _ = yes (left a)
_ ∨? yes b = yes (right b)
no ¬a ∨? no ¬b = no λ x → case x (λ a → ¬a a) (λ b → ¬b b)

### Law of excluded middle

Another way to look at decidable properties is through the law of the excluded middle. In fact, decidable properties are designed to be precisely those for which the law of excluded middle holds.

**Exercise.** Prove that a property is decidable iff it satisfy the law of the excluded middle.

In [ ]:
module code.decidable.lem where
open import code.decidable public

dec→lem : {A : Set} → Dec A → A ∨ ¬ A
dec→lem = ?

lem→dec : {A : Set} → A ∨ ¬ A → Dec A
lem→dec = ?

?0 : Dec A → A ∨ ¬ A
?1 : A ∨ ¬ A → Dec A


In [ ]:
module code.decidable.lem where
open import code.decidable public

dec→lem : {A : Set} → Dec A → A ∨ ¬ A
dec→lem (yes a) = left a
dec→lem (no ¬a) = right ¬a

lem→dec : {A : Set} → A ∨ ¬ A → Dec A
lem→dec (left a) = yes a
lem→dec (right ¬a) = no ¬a

### Quantifiers

````
module code.decidable.quantifiers where
open import code.decidable.logic public
````

One may ask whether decidable predicates are closed w.r.t. existential/universal quantification, but this is not the case.

Closure under the existential quantifier can be stated as follows:
Assume that `B a` is decidable for every fixed `a`
and show that `∃[ a ] B a` must be decidable.

````
∃? : {A : Set} {B : A → Set} → (∀[ a ] Dec (B a)) → Dec (∃[ a ] B a)
∃? _ = ?
````

Even if we can decide `B a` for a fixed `a`, 
it is not clear

1. how to find such an `a` when it exists, and
2. what to do when no such `a` exists.

This is in line with the fact that recursively enumerable sets are projections of recursive sets (decidables).

The situation for the universal quantifier is dually complicated.

````
∀? : {A : Set} {B : A → Set} → (∀[ a ] Dec (B a)) → Dec (∀[ a ] B a)
∀? _ = ?
````

# Type-level programming

## Two-counter machines

````
module code.minsky where
open import code.nat public
````

We show how to implement a two-counter machine with zero tests (i.e., a Minsky machine) with inductive types and type-level programming.
This shows that 

First of all, we define the set of allower counter operations: 

````
data Op : Set where
  nop : Op
  incr1 : Op
  decr1 : Op
  test1 : Op
  incr2 : Op
  decr2 : Op
  test2 : Op
````

For instance, `nop` does not change the counters and `incr2` increases the second counter by one.

We also define the set of states (i.e., control locations) of our machine.

````
data State : Set where
  p : State
  q : State
  r : State
````

Thus our machine has three states.
We can now define the set of transitions.

````
data Δ : State → Op → State → Set where
  δ0 : Δ p nop p
  δ1 : Δ p incr1 p
  δ2 : Δ p test2 q
  δ3 : Δ q decr1 r
````

For instance, `δ2` can be applied when the machine is in state `p` and the second counter is zero,
and causes the machine to go to state `q`.

We are now ready to formally define the semantics of counter machines.
A *configuration* is a tuple `[ p , c , d ]` where `p` is the current control location,
and `c`, `d` are the current counter values.

````
data Conf : Set where
  [_,_,_] : State → ℕ → ℕ → Conf
````

The transition relation defines the dynamics of counter machines in terms of a transition system.

````
data _=>_ : Conf → Conf → Set
````

In the definition of `_=>_` we use the following type-level function
which maps counter operations and states to a corresponding transition.

````
⟬_⟭ : Op → State → State → Set
⟬ nop ⟭ x y = ∀ {m n} → [ x , m , n ] => [ y , m , n ]
⟬ incr1 ⟭ x y = ∀ {m n} → [ x , m , n ] => [ y , suc m , n ]
⟬ decr1 ⟭ x y = ∀ {m n} → [ x , suc m , n ] => [ y , m , n ]
⟬ test1 ⟭ x y = ∀ {n} → [ x , zero , n ] => [ y , zero , n ]
⟬ incr2 ⟭ x y = ∀ {m n} → [ x , m , n ] => [ y , m , suc n ]
⟬ decr2 ⟭ x y = ∀ {m n} → [ x , m , suc n ] => [ y , m , n ]
⟬ test2 ⟭ x y = ∀ {m} → [ x , m , zero ] => [ y , m , zero ]
````

For instance, `⟬ decr2 ⟭ x y` says that from configuration `[ x , m , suc n ]`
we can go to configuration `[ y , m , n ]`, by decrementing the second counter.

The definition of the transition relation leverages on `⟬_⟭`.

````
data _=>_ where
    step-nop : ∀ {x y} → Δ x nop y → ⟬ nop ⟭ x y
    step-incr1 : ∀ {x y} → Δ x incr1 y → ⟬ incr1 ⟭ x y
    step-decr1 : ∀ {x y} → Δ x decr1 y → ⟬ decr1 ⟭ x y
    step-test1 : ∀ {x y} → Δ x test1 y → ⟬ test1 ⟭ x y
    step-incr2 : ∀ {x y} → Δ x incr2 y → ⟬ incr2 ⟭ x y
    step-decr2 : ∀ {x y} → Δ x decr2 y → ⟬ decr2 ⟭ x y
    step-test2 : ∀ {x y} → Δ x test2 y → ⟬ test2 ⟭ x y
````

The transition relation describes runs of length 1.
In order to describe longer run,
we take its reflexive-transitive closure,
which we define in a generic way.
This is the only place where we use inductive types.

````
-- careful, the symbol ";" below looks like ";" but it is a different unicode symbol ;p
infix 30 _*
infixr 20 _;_
data _* {A : Set} (R : A → A → Set) : A → A → Set where
  stop : ∀ {a} → (R *) a a
  _;_ : ∀ {a b c} → R a b → (R *) b c → (R *) a c
````

We can now prove the existence of some runs, such as:

````
_ : (_=>_ *) [ p , zero , zero ] [ r , zero , zero ]
_ = step-incr1 δ1 ; step-test2 δ2 ; step-decr1 δ3 ; stop
````

Note that once we know the transition `δ2`,
then the corresponding constructor `step-test2` of `_=>_` is uniquely determined thanks to the type system.
We formalise below this intuition.

````
step : ∀ {x y op} → Δ x op y → ⟬ op ⟭ x y
step {op = nop} δ = step-nop δ
step {op = incr1} δ = step-incr1 δ
step {op = decr1} δ = step-decr1 δ
step {op = test1} δ = step-test1 δ
step {op = incr2} δ = step-incr2 δ
step {op = decr2} δ = step-decr2 δ
step {op = test2} δ = step-test2 δ
````

With the generic `step` function in hand,
we can now reprove the existence of the previous run more elegantly.

````
_ : (_=>_ *) [ p , zero , zero ] [ r , zero , zero ]
_ = step δ1 ; step δ2 ; step δ3 ; stop
````

Since the reachability problem for Minsky machines is undecidable,
it follows that the inhabitation problem for types of the form `(_=>_ *) s t` is also undecidable.

# Simply typed lambda calculus

````
module code.lambda where
open import code.decidable public
open import code.eq public
open import code.nat public
open import code.list public
open import code.existential public
````

## Simple types
We begin by defining the set of simple types:
A simple type is either the base type `o`,
or it is a function type `α ⇒ β`,
where `α`, `β` are base types.

````
infixr 30 _⇒_
data Type : Set where
  o : Type
  _⇒_ : Type → Type → Type
````

## Decidable equality

We show that simple types have a decidable equality relation.

````
inversion : ∀ {α β σ τ} → α ⇒ σ ≡ β ⇒ τ → α ≡ β ∧ σ ≡ τ
inversion refl = ⟨ refl , refl ⟩

_≡?_ : (α β : Type) → Dec (α ≡ β)
o ≡? o = yes refl
o ≡? (_ ⇒ _) = no λ ()
(_ ⇒ _) ≡? o = no λ ()
(α ⇒ α₁) ≡? (β ⇒ β₁) with α ≡? β | α₁ ≡? β₁
... | yes α≡β | yes α₁≡β₁ = yes (cong2 _⇒_ α≡β α₁≡β₁)
... | no ¬α≡β | _ = no λ x → ¬α≡β (fst (inversion x))
... | _ | no ¬α₁≡β₁ = no λ x → ¬α₁≡β₁ (snd (inversion x))
````

For the negative case, we use the `inversion` property:
If two functional types are equal, then their components must be equal as well.

## Raw terms

We define the set of raw terms.
Variables are indexed by their de Bruijn index.
We additionally decorate λ-abstractions with the type of the bound variable
(types *à la Church*).

````
infixl 80 _·_
data Raw : Set where
  var : ℕ → Raw
  _·_ : Raw → Raw → Raw
  lam : Type → Raw → Raw
````

For instance, the term `λ f λ x → f x` is represented as follows.

````
u : Raw
u = lam (o ⇒ o) (lam o (var 1 · var 0))
````

````
·-≡ : ∀ {u v u' v'} → u · v ≡ u' · v' → u ≡ u' ∧ v ≡ v'
·-≡ refl = ⟨ refl , refl ⟩

lam-≡ : {α β : Type} {u v : Raw} → lam α u ≡ lam β v → α ≡ β ∧ u ≡ v
lam-≡ refl = ⟨ refl , refl ⟩
````

## Contexes

A context is a mapping assigning a type to every variable.
Since variables are indexed by natural numbers,
a context is just a list of types.

It is customary to write contexes right-to-left,
with the first element appearing at the right.

````
infixl 5 _,_
data Context : Set where
  ∅ : Context
  _,_ : Context → Type → Context
````

For instance, the following context assigns type `o` to the innermost variable
and type `o ⇒ o` to the second innermost one.

````
Γ₀ : Context
Γ₀ = ∅ , o ⇒ o , o
````

## Variable lookup

We define when a type appears in a context.
We additionally record at which index does the occurrence appear.

````
infix  4 _∈_!_
data _∈_!_ : Type → Context → ℕ → Set where
  stop : ∀ {Γ α} → α ∈ Γ , α ! 0
  skip : ∀ {Γ α β n} → α ∈ Γ ! n → α ∈ Γ , β ! suc n
````

For example,

````
_ : o ⇒ o ∈ Γ₀ ! 1
_ = skip stop
````

The constructor `stop` tells us how to prove a lookup of the form `α ∈ Γ , α ! 0`.
In fact, there is no other way, as show below.

````
stop-inversion : ∀ {Γ α β} → α ∈ Γ , β ! 0 → α ≡ β
stop-inversion stop = refl
````

A similar property holds for `skip`:
The only way to prove `α ∈ Γ , β ! suc n` is by proving `α ∈ Γ ! n` as well.

````
skip-inversion : ∀ {Γ α β n} → α ∈ Γ , β ! suc n → α ∈ Γ ! n
skip-inversion (skip x) = x
````

Finally, we can show that lookup is deterministic,
which will be useful later.

````
∈-det : ∀ {Γ α β n} → α ∈ Γ ! n → β ∈ Γ ! n → α ≡ β
∈-det stop stop = refl
∈-det (skip p) (skip q) with ∈-det p q
... | refl = refl
````

## Decidable lookup

When `α ∈ Γ ! n` holds we can immediately recover the index of the first occurrence of `α` in `Γ`.

````
index : ∀ {Γ α n} → α ∈ Γ ! n → ℕ
index {n = n} _ = n
````

We can also go the other way around,
which amounts to prove that variable lookup is a decidable property.
The `yes` cases are immediate. In the last `no` case,
we use the inversion property `skip-inversion` above.

<!--
_∈?_!_ : ∀ α Γ n → Dec (α ∈ Γ ! n)
_ ∈? ∅ ! _ = no λ ()
α ∈? Γ , β ! zero with α ≡? β
... | yes refl = yes stop
... | no ¬α≡β = no λ x → ¬α≡β (stop-inversion x)
α ∈? Γ , β ! suc n with α ∈? Γ ! n
... | yes p = yes (skip p)
... | no p = no λ x → p (skip-inversion x)
-->

````
infix 4 _!_
_!_ : ∀ Γ n → Dec (∃[ α ] α ∈ Γ ! n)
∅ ! _ = no λ ()
Γ , α ! zero = yes ⟨ α , stop ⟩
Γ , _ ! suc n with Γ ! n
... | yes ⟨ α , α∈Γ!n ⟩ = yes ⟨ α , skip α∈Γ!n ⟩
... | no f = no λ x → f ⟨ Σ.proj₁ x , skip-inversion (Σ.proj₂ x) ⟩
````

## Typed terms

Typed terms are identified with typing derivations.
The syntax mirrors the one for raw terms.

````
data Term : Context → Type → Set where
  var : ∀ {Γ α n} → α ∈ Γ ! n → Term Γ α
  _·_ : ∀ {Γ α β} → Term Γ (α ⇒ β) → Term Γ α → Term Γ β
  lam : ∀ {Γ β} α → Term (Γ , α) β → Term Γ (α ⇒ β)
````

We can easily convert a typed term into its correspoding raw term
just by discarding the typing information.
We use the function `index` to convert evidence that `α ∈ Γ` holds
into the index of the occurrence of `α` in `Γ`.

````
erase : ∀ {Γ α} → Term Γ α → Raw
erase (var A∈Γ) = var (index A∈Γ)
erase (u · v) = erase u · erase v
erase (lam α u) = lam α (erase u)
````

We can show that `erase` preserves the type, in the following sense:
If two typed terms `s` and `t` have the same erasure,
then they have the same type.

````
type-det : ∀ {Γ α β} → (s : Term Γ α) → (t : Term Γ β) → erase s ≡ erase t → α ≡ β
type-det (var x) (var y) refl = ∈-det x y
type-det (s · t) (s' · t') eq with ·-≡ eq
... | ⟨ eq1 , eq2 ⟩ with type-det s s' eq1
... | refl = refl
type-det (lam α s) (lam β t) eq with lam-≡ {α} {β} {erase s} {erase t} eq
... | ⟨ refl , eq2 ⟩ with type-det s t eq2
... | refl = refl
````

## Typable terms

We define raw typable terms as erasures of typed terms.

````
infix 4 _⊢_
data _⊢_ (Γ : Context) : Raw → Set where
  typable : (α : Type) → (t : Term Γ α) → Γ ⊢ erase t
````

The typability relation also satisfies an inversion property.

````
typable-inversion : ∀ {Γ u} → Γ ⊢ u → ∃[ α ] Σ (Term Γ α) (λ t → u ≡ erase t)
typable-inversion (typable α t) = ⟨ α , ⟨ t , refl ⟩ ⟩
````

## Properties of `_⊢_`

We cannot type a variable from the empty context.

````
∅-var : ∀ {n} → ¬ (∅ ⊢ var n)
∅-var ∅⊢var with typable-inversion ∅⊢var
... | ⟨ _ , ⟨ var () , _ ⟩ ⟩
... | ⟨ _ , ⟨ u · v , () ⟩ ⟩
... | ⟨ _ , ⟨ lam α t , () ⟩ ⟩
````

### Variables
If we can type a raw term of the form `var n`,
then its type is to be found at index `n` in `Γ`.

````
var-⊢ : ∀ {Γ n} → Γ ⊢ var n → ∃[ α ] α ∈ Γ ! n 
var-⊢ {∅} {_} x = ⊥-elim (∅-var x)
var-⊢ {Γ , α} {0} x = ⟨ α , stop ⟩
var-⊢ {Γ , α} {suc n} x with typable-inversion x 
... | ⟨ β , ⟨ var (skip p) , refl ⟩ ⟩ = ⟨ β , skip p ⟩
````

If we can type a raw term of the form `lam α u` in context `Γ`,
then we can type `u` in context `Γ , α`.

### Abstractions

````
lam-⊢ : ∀ {Γ α u} → Γ ⊢ lam α u → Γ , α ⊢ u
lam-⊢ x with typable-inversion x
... | ⟨ _ , ⟨ lam α t , refl ⟩ ⟩ = typable _ t
````

### Applications

If we can type a raw term of the form `u · v` in context `Γ`,
then it must be the case that both `u` and `v` are typable in context `Γ`.

````
app-⊢ : ∀ {Γ u v} → Γ ⊢ u · v → Γ ⊢ u ∧ Γ ⊢ v
app-⊢ x with typable-inversion x
... | ⟨ _ , ⟨ s · t , refl ⟩ ⟩ = ⟨ typable _ s , typable _ t ⟩
````

In an application `r · v`, the typed term `r` must have an arrow type.

````
app-type1 : ∀ {Γ v α} → (r : Term Γ α) → Γ ⊢ erase r · v → ¬ (α ≡ o)
app-type1 {Γ} {v} {α} r x α≡o with typable-inversion x
... | ⟨ β , ⟨ s · t , p ⟩ ⟩ with subst (_≡ o) (type-det r s (fst (·-≡ p))) α≡o
... | ()
````

In an application `erase s · erase t` where `t` has type `α`
the type of `s` must be of the form `α ⇒ β`.

````
app-type2 : ∀ {Γ α β γ} → (s : Term Γ (α ⇒ β)) → (t : Term Γ γ) → Γ ⊢ erase s · erase t → α ≡ γ
app-type2 s t x with typable-inversion x
... | ⟨ β , ⟨ y · z , p ⟩ ⟩ with ·-≡ p
... | ⟨ eq1 , eq2 ⟩ with inversion (type-det s y eq1)
... | ⟨ eq3 , eq4 ⟩ with type-det t z eq2
... | eq5 = trans eq3 (sym eq5)
````

## Type inference

We are now ready to present the type inference algorithm.
It amounts to establishing that the relationship `Γ ⊢ u`
between contexes and raw terms is decidable.

````
infix 4 _⊢?_
_⊢?_ : ∀ Γ u → Dec (Γ ⊢ u)
````

Let `Γ ⊢? u` be an instance of typability.
In the `yes` instances, we return the type of `u`, say `α`,
together with evidence of the form `typable α s`
where `s` is a typed term whose is erasure is `u`.
This provides the proof of *soundness* of the typing algorithm.

In the `no` instances,
we return evidence that no typing possibly exists.
This provides the proof of *completeness* of the typing algorithm.

The variable and abstraction cases are the easiest.
The inversion properties `var-⊢` and `lam-⊢` introduced earlier
are used to conclude in the `no` instances.

````
Γ ⊢? var n
    with Γ ! n
... | yes ⟨ α , α∈Γ!n ⟩ = yes (typable α (var α∈Γ!n))
... | no f = no λ x → f (var-⊢ x)

Γ ⊢? lam α u with Γ , α ⊢? u
... | yes (typable β s) = yes (typable (α ⇒ β) (lam α s))
... | no f = no λ x → f (lam-⊢ x)
````

The application case is the most involved,
because there are many things that can go wrong in an application,
and we need to analyse them one by one.

````
Γ ⊢? u · v
    with Γ ⊢? u | Γ ⊢? v
... | no f | _ = no λ x → f (fst (app-⊢ x))
... | _ | no f = no λ x → f (snd (app-⊢ x))
... | yes (typable o s) | _ = no λ x → (app-type1 s x) refl
... | yes (typable (α ⇒ β) s) | yes (typable γ t)
    with α ≡? γ
Γ ⊢? .(erase s) · .(erase t) | yes (typable (α ⇒ β) s) | yes (typable .α t) | yes refl = yes (typable β (s · t))
Γ ⊢? u · v | yes (typable (α ⇒ β) s) | yes (typable γ t) | no f = no λ x → f (app-type2 s t x)
````

# Gödel System T

````
module code.system-T where
open import code.boolean public
open import code.nat public

if_then_else_ : {C : Set} → 𝔹 → C → C → C
if true then x else y = x
if false then x else y = y

natrec : {C : Set} → C → (ℕ → C → C) → ℕ → C
natrec p h zero = p
natrec p h (suc n) = h n (natrec p h n)

plus : ℕ → ℕ → ℕ
plus n m = natrec m (\x y -> suc y) n

mult : ℕ → ℕ → ℕ
mult n m = natrec zero (\x y -> plus y m) n
````

Features:

* All typable programs terminate (even at higher order).
* It is even *strongly normalising*, i.e., a typable term reaches a unique normal form, no matter the reduction strategy.
* We can define all primitive higher-order recursive functions.
In particular, we can define all primitive (first-order) recursive functions.
Primitive higher-order recursion allows us even to define functions which are not primitive first-order, such as the Ackermann function (!).
* System T can be seen as the simply typed core of Martin-Löf's constructive type theory.